In [249]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# TitanicDataset 클래스는 PyTorch의 Dataset을 상속받음
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        """
        TitanicDataset의 생성자.
    
        :param X (numpy 배열 또는 DataFrame): 특성 행렬 (입력 데이터).
        :param y (numpy 배열 또는 DataFrame): 레이블 (타겟 데이터).
        """
        self.X = torch.FloatTensor(X)  # 특성을 FloatTensor로 변환
        self.y = torch.LongTensor(y)    # 타겟 값을 LongTensor로 변환

    def __len__(self):
        """
        :return: 
            int: 데이터의 총 샘플 수 반환
        """
        return len(self.X)  # 데이터의 총 샘플 수 반환

    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 샘플을 반환.
        
        :param idx: (int) 인덱스 번호.
        :return:
            dict: {'input': 특성, 'target': 타겟 값} 형태로 반환. 
        """
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    def __str__(self):
        """
        데이터셋의 크기와 입력, 타겟의 형태를 문자열로 반환.
        
        :return: 
            str: 데이터셋의 크기,  입력 데이터, 타겟 데이터의 형태를 나타내는 문자열. 
        """
        str = "데이터 크기: {0}, 입력 형태: {1}, 타겟 형태: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )
        return str

### TitanicDataset class 생성
* PyTorch의 Dataset 클래스 상속
* 데이터 타입 변환: Titanic 데이터를 PyTorch 모델 학습에 적합한 형태로 만듦
* 인덱싱기능 제공
* DataLoader과의 통합 가능

In [250]:
from torch.utils.data import Dataset

# TitanicTestDataset 클래스는 테스트용 데이터셋을 다루기 위한 PyTorch의 Dataset을 상속받음
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        """
        TitanicTestDataset의 생성자.

        :param X (numpy 배열 또는 DataFrame): 특성 행렬 (입력 데이터).

        입력 특성을 PyTorch 텐서로 변환.
        """
        self.X = torch.FloatTensor(X)  # 특성을 FloatTensor로 변환

    def __len__(self):
        """
        데이터셋의 총 샘플 수를 반환.
    
        :return:
            int: 데이터셋의 샘플 수.
        """
        return len(self.X)

    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 샘플을 반환.
        
        :param idx: (int) 인덱스 번호.

        :return:
            dict: {'input': 특성} 형태로 반환.
        """
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        """
        데이터셋의 크기와 입력의 형태를 문자열로 반환.
        
        :return:
            str: 데이터셋의 크기,  입력 데이터
        """
        str = "데이터 크기: {0}, 입력 형태: {1}".format(
            len(self.X), self.X.shape
        )
        return str


### target이 없는 이유
* TitanicDataset는 학습용 데이터를 다루기 위한 클래스
* TitanicTestDataset는 테스트용 데이터를 다루기 위한 클래스
* 타겟 데이터를 예측해야 하기에 입력데이터만 주어짐

In [251]:
# 전처리된 데이터셋을 생성하는 함수
def get_preprocessed_dataset():
    # 현재 파일 경로를 가져옴
    CURRENT_FILE_PATH = os.getcwd()

    # 학습 데이터와 테스트 데이터 파일 경로를 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # 학습 데이터와 테스트 데이터를 읽어옴
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 학습 데이터와 테스트 데이터를 하나로 결합 (sort=False로 기존 순서 유지)
    all_df = pd.concat([train_df, test_df], sort=False)

    # 첫 번째 전처리 단계 적용
    all_df = get_preprocessed_dataset_1(all_df)
    
    # 두 번째 전처리 단계 적용
    all_df = get_preprocessed_dataset_2(all_df)

    # 세 번째 전처리 단계 적용
    all_df = get_preprocessed_dataset_3(all_df)

    # 네 번째 전처리 단계 적용
    all_df = get_preprocessed_dataset_4(all_df)

    # 다섯 번째 전처리 단계 적용
    all_df = get_preprocessed_dataset_5(all_df)

    # 여섯 번째 전처리 단계 적용
    all_df = get_preprocessed_dataset_6(all_df)

    # 'Survived' 값이 존재하는 행은 학습 데이터로 사용하고, 'Survived' 열을 제거
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 학습 데이터의 레이블(Survived 값)을 설정
    train_y = train_df["Survived"]

    # 'Survived' 값이 없는 행은 테스트 데이터로 사용하고, 'Survived' 열을 제거
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 학습 데이터셋 생성 (TitanicDataset 클래스 사용)
    dataset = TitanicDataset(train_X.values, train_y.values)

    # 학습 데이터셋을 80%는 학습용, 20%는 검증용으로 분할
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])

    # 테스트 데이터셋 생성 (TitanicTestDataset 클래스 사용)
    test_dataset = TitanicTestDataset(test_X.values)

    # 전처리된 학습, 검증, 테스트 데이터셋 반환
    return train_dataset, validation_dataset, test_dataset

### 학습용 데이터셋과 테스트 데이터셋 생성 과정
1. 데이터 파일 경로 및 파일 읽기
2. 데이터 병합 
    * pd.concat()를 사용하여 학습 데이터와 테스트 데이터를 결합,
    * sort=False를 사용하여 기존 데이터 순서를 유지하면서 병합
3. 데이터 전처리 과정 관리
4. 학습/검증/테스트 데이터 분리
    * 학습데이터는 train_x 값이 Survived 값이 존재
    * 테스트데이터는 test_x는 Survived 값이 없음
5. Dataset 클래스 사용
6. 학습 데이터와 검증 데이터로 나누기
    * random_split()로 8대2로 나눔
7. 전처리된 데이터셋의 반환
8. 데이터 리셋 및 정리
    * reset_index(drop=True)로 인덱스 재설정


In [252]:
def get_preprocessed_dataset_1(all_df):
    """
    전처리 단계 1: Pclass(티켓 등급)별로 Fare(요금)의 평균값을 사용하여 결측값을 메우는 함수.

    :param all_df: (DataFrame) 결합된 학습 및 테스트 데이터셋.

    :return: 
        all_df: (DataFrame) Fare 결측값을 Pclass별 평균으로 채운 후 반환된 데이터프레임.
    """
    
    # Pclass별 Fare 평균값 계산
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()

    # 새로운 열 이름을 "Pclass", "Fare_mean"으로 설정
    Fare_mean.columns = ["Pclass", "Fare_mean"]

    # 원본 데이터프레임에 Pclass별 평균 Fare 값을 병합 (left join)
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")

    # Fare 값이 결측된 행에 대해, Pclass별 평균 Fare 값으로 채움
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    # 전처리 후의 데이터프레임 반환
    return all_df


### Fare 결측값을 채우기
* 티켓 등급별 요금의 평균 값을 구해서 df를 갱신한다

In [253]:
def get_preprocessed_dataset_2(all_df):
    """
    전처리 단계 2: 승객의 이름(Name)을 세 개의 컬럼(family_name, honorific, name)으로 분리하는 함수.

    :param all_df: (DataFrame) 결합된 학습 및 테스트 데이터셋.
    
    :return:
        all_df: (DataFrame) 이름 정보를 분리하여 새로운 컬럼들을 추가한 데이터프레임.
    """
    
    # Name 열을 쉼표(,)와 마침표(.)를 기준으로 세 부분으로 분리
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)

    # 분리된 열에 새로운 이름(family_name, honorific, name) 부여
    name_df.columns = ["family_name", "honorific", "name"]

    # 각 열의 값에 있는 공백을 제거
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()

    # 원본 데이터프레임에 분리된 이름 정보 열을 추가 (axis=1로 수평 방향으로 결합)
    all_df = pd.concat([all_df, name_df], axis=1)

    # 전처리 후의 데이터프레임 반환
    return all_df


### 이름 분리하기
* 이름 정보를 분리하고 공백을 없애 데이터를 일반화 시킴

In [254]:
def get_preprocessed_dataset_3(all_df):
    """
    전처리 단계 3: 경칭(honorific)별로 나이(Age)의 중앙값을 사용하여 결측값을 메우는 함수.

    :param all_df: (DataFrame) 결합된 학습 및 테스트 데이터셋.

    Returns:
    all_df: (DataFrame) Age 결측값을 honorific별 중앙값으로 채운 후 반환된 데이터프레임.
    """
    
    # honorific별로 Age의 중앙값을 계산하여 DataFrame으로 저장
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()

    # 새로운 열 이름을 "honorific", "honorific_age_mean"으로 설정
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]

    # 원본 데이터프레임에 honorific별 중앙값을 병합 (left join)
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")

    # Age 값이 결측된 경우 honorific별 중앙값으로 채움
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]

    # 결측값을 채운 후, 사용했던 honorific_age_mean 열을 삭제
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    # 전처리 후의 데이터프레임 반환
    return all_df


### 나이 결측값 채우기
* 경칭 별로 나이의 중앙값을 구해 결측값을 채운다

In [255]:
def get_preprocessed_dataset_4(all_df):
    """
    전처리 단계 4: 가족 수와 혼자 탑승 여부를 나타내는 새로운 컬럼을 추가하고, 불필요한 컬럼을 제거하는 함수.

    :param all_df: (DataFrame) 결합된 학습 및 테스트 데이터셋.

    :return:
        all_df: (DataFrame) 가족 수(family_num)와 혼자 탑승 여부(alone)를 추가하고 불필요한 컬럼을 제거한 데이터프레임.
    """
    
    # 가족 수(family_num) 계산: Parch(부모/자식 수) + SibSp(형제/배우자 수)
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자 탑승 여부(alone) 컬럼 추가: 가족 수가 0이면 혼자, 그렇지 않으면 함께 탑승
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1  # 가족 수가 0인 경우 alone을 1로 설정
    all_df["alone"] = all_df["alone"].fillna(0)
  # 그 외 경우는 혼자가 아니므로 alone을 0으로 설정

    # 학습에 불필요한 컬럼 제거: PassengerId, Name, family_name, name, Ticket, Cabin
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    # 전처리 후의 데이터프레임 반환
    return all_df


### 가족 수와 혼자탑승여부 열 추가, 불필요한 열 제거
* 가족 수가 0인 경우 혼자 탑승여부 = True

In [256]:
def get_preprocessed_dataset_5(all_df):
    """
    전처리 단계 5: honorific(경칭) 값을 줄이고, Embarked(탑승 항구) 결측값을 처리하는 함수.

    :param all_df: (DataFrame): 결합된 학습 및 테스트 데이터셋.

    :return:
        all_df: (DataFrame) honorific 값을 주요 네 개(Mr, Miss, Mrs, Master)로 축소하고 나머지는 'other'로 대체한 후, Embarked 결측값을 'missing'으로 처리한 데이터프레임.
    """
    
    # honorific 값이 Mr, Miss, Mrs, Master가 아닌 경우 모두 'other'로 설정
    all_df.loc[
        ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
        ),
        "honorific"
    ] = "other"

    # Embarked 값이 결측인 경우 'missing'으로 채움
    all_df["Embarked"] = all_df["Embarked"].fillna("missing")

    # 전처리 후의 데이터프레임 반환
    return all_df


### 경칭 값과 탑승항구 데이터 수정
* 경칭을 대표적인 사례 4개로 줄임
* 탑승 항구의 결측값 수정

In [257]:
def get_preprocessed_dataset_6(all_df):
    """
    전처리 단계 6: 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변환하는 함수.

    :param all_df: (DataFrame): 결합된 학습 및 테스트 데이터셋.

    :return:
        all_df: (DataFrame) 카테고리 변수가 수치값으로 변환된 데이터프레임.
    """
    
    # 카테고리 변수(객체형 데이터 타입) 목록 가져오기
    category_features = all_df.columns[all_df.dtypes == "object"]
    
    from sklearn.preprocessing import LabelEncoder
    
    # 각 카테고리 변수를 수치값으로 변환
    for category_feature in category_features:
        le = LabelEncoder()  # LabelEncoder 객체 생성
        if all_df[category_feature].dtypes == "object":  # 데이터 타입이 객체인 경우
            le = le.fit(all_df[category_feature])  # 해당 카테고리 변수에 대해 LabelEncoder 학습
            all_df[category_feature] = le.transform(all_df[category_feature])  # 변환된 값을 원래 데이터프레임에 저장

    # 전처리 후의 데이터프레임 반환
    return all_df


### 카테고리 변수를 수치값으로 변환
* 범주형 데이터를 수치형으로 바꿔야함
* Label Encoding으로 변환
* 객체형 데이터 자동 탐지
* 독립적인 인코더 사용

In [258]:
from torch import nn

class testModel(nn.Module):
    """
    사용자 정의 신경망 모델 클래스.
    """
    def __init__(self, n_input, n_output):
        """
        신경망 모델을 초기화하는 생성자 함수.
        입력층, 은닉층, 출력층을 Sequential 컨테이너에 정의합니다.

        :param n_input: (int) 입력층의 특성 수 (입력 데이터의 크기).
        :param n_output: (int) 출력층의 특성 수 (예측해야 할 클래스 또는 값의 크기).
        """
        # 부모 클래스 초기화
        super().__init__()

        # 신경망 모델 구성: Sequential 컨테이너에 층 추가
        self.model = nn.Sequential(
            nn.Linear(n_input, 30),  # 입력층: 입력 특성 수에서 30개의 뉴런으로 변환
            nn.ReLU(),                # 활성화 함수: ReLU
            nn.Linear(30, 30),        # 은닉층: 30개의 뉴런을 가지는 두 번째 층
            nn.ReLU(),                # 활성화 함수: ReLU
            nn.Linear(30, n_output),  # 출력층: 30개의 뉴런에서 출력 특성 수로 변환
        )

    def forward(self, x):
        """
        순전파 메서드: 입력 데이터를 모델에 통과시켜 출력 생성.
        
        :param x: (Tensor) 입력 데이터.
        
        :return:
            Tensor: 모델의 출력.
        """
        x = self.model(x)  # 입력 데이터를 모델에 통과
        return x  # 출력 반환

### 모델 생성
1. PyTorch 모델 정의
* 사용자 정의 모델 클래스: PyTorch에서 `nn.Module`을 상속받아 새로운 클래스를 정의.
2. 모델의 계층 구성
* `nn.Sequential` 사용: PyTorch의 `nn.Sequential`을 사용하여 신경망의 여러 계층을 순차적으로 쌓음. 
* `nn.Linear`: 선형 변환을 정의하는 `nn.Linear`의 사용하여 입력 크기와 출력 크기를 지정하여 각 계층을 구성.
* 예: `nn.Linear(n_input, 30)`는 입력 데이터를 30개의 뉴런으로 변환하는 층을 정의.
* 활성화 함수: `nn.ReLU` 같은 활성화 함수를 사용하여 계층의 출력에 비선형성을 부여.
3. 순전파(Forward Propagation)
* `forward()` 메서드: PyTorch에서 신경망의 순전파 과정을 정의
* 이 메서드는 입력 데이터를 신경망에 통과시켜 최종 출력을 생성하는 과정을 구현.
* 입력 데이터 처리: `forward()` 메서드 내에 입력 데이터를 `self.model(x)`처럼 정의된 모델에 입력하여 순차적으로 계층을 통과.
4. 하이퍼파라미터 관리
* 입력 크기와 출력 크기 관리: `n_input`과 `n_output`을 통해 모델의 입력과 출력 크기를 유연하게 정의
5. 활성화 함수 사용
* ReLU 활성화 함수: `ReLU()`는 뉴런의 출력 값이 0 이상이면 그대로 반환, 0보다 작으면 0으로 만드는 활성화 함수.
* 활성화 함수는 각 계층 사이에 추가되어 비선형성을 제공, 이 코드는 ReLU를 은닉층 사이에 적용하여 입력-출력 사이의 관계를 비선형적으로 변환합니다.

In [259]:
def Dtest(test_data_loader):
    """
    테스트 함수: 주어진 테스트 데이터 로더를 사용하여 모델의 예측을 수행하고 결과를 출력합니다.

    :param test_data_loader: (DataLoader) 테스트 데이터셋을 로드하는 DataLoader 객체.
    """
    print("[TEST]")  # 테스트 시작 알림
    
    # 테스트 데이터 로더에서 다음 배치를 가져오기
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))  # 입력 배치의 크기 출력
    
    # MyModel 인스턴스 생성: 입력 특성 수는 11, 출력 클래스 수는 2
    my_model = testModel(n_input=11, n_output=2)
    
    # 모델을 사용하여 배치 입력으로부터 출력 생성
    output_batch = my_model(batch['input'])
    
    # 출력에서 가장 높은 값을 가지는 인덱스(예측 클래스)를 찾음
    prediction_batch = torch.argmax(output_batch, dim=1)
    
    # 각 예측 결과를 인덱스와 함께 출력
    for idx, prediction in enumerate(prediction_batch, start=892):
        if (idx - 892) % 10 == 0:
            print(f"Index: {idx}, Prediction: {prediction.item()}")


### test() 함수 생성
1. DataLoader의 역할
* `test_data_loader` : PyTorch의 `DataLoader`로, 테스트 데이터를 배치 단위로 로드.
* `next(iter(test_data_loader))` : 첫 번째 배치를 가져와 모델의 입력데이터로 사용 가능한 배치 제공
2. 모델 생성 및 예측
* `MyModel(n_input=11, n_output=2)`: 테스트 중 사용할 모델을 생성하는 과정에서, 입력 특성 수와 출력 클래스 수를 설정합니다.
* `n_input=11`: 모델이 입력으로 받는 특성의 수 (예: Titanic 데이터의 경우 승객의 여러 특성).
* `n_output=2`: 예측하려는 클래스 수 (예: 생존 여부를 예측하는 이진 분류 문제).
* 모델 생성 후 순전파를 통해 입력 데이터(batch['input'])를 모델에 넣고, 출력을 얻음.
4. 예측 클래스 추출
* `torch.argmax(output_batch, dim=1)`: 모델의 출력 중 가장 높은 확률을 가진 클래스를 예측 값으로 추출
* `torch.argmax()`는 예측된 클래스 확률 중에서 가장 큰 값을 가진 클래스의 인덱스를 반환
5. 배치 크기 확인
* `batch['input'].shape`: 테스트 데이터의 배치 크기 및 입력 데이터의 형태를 확인
6. 결과 출력
* `enumerate()`: 배치에서 각 샘플의 인덱스와 예측값을 순서대로 출력
* `prediction.item()`: torch.Tensor의 값을 Python 스칼라 값으로 변환하여 예측된 클래스를 출력

In [260]:
if __name__ == "__main__":
    """
    메인 프로그램: 데이터셋을 전처리하고, 데이터로더를 생성하며,
    학습 및 검증 데이터셋을 출력하고 테스트를 수행합니다.
    """

    # 전처리된 데이터셋 가져오기
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    # 데이터셋의 크기 출력
    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)  # 구분선 출력

    # 학습 데이터셋의 각 샘플을 출력
    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)  # 구분선 출력

    # DataLoader 생성: 배치 크기 16, 데이터 셔플링 활성화
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))  # 테스트는 전체 배치로

    print("[TRAIN]")  # 학습 데이터 출력 시작
    # 학습 데이터 로더에서 배치 가져오기 및 출력
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("[VALIDATION]")  # 검증 데이터 출력 시작
    # 검증 데이터 로더에서 배치 가져오기 및 출력
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)  # 구분선 출력

    # 테스트 수행
    Dtest(test_data_loader)


train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 20.0000,  0.0000,  0.0000,  9.8458,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
1 - tensor([ 1.0000,  1.0000, 54.0000,  0.0000,  1.0000, 77.2875,  2.0000, 87.5090,
         2.0000,  1.0000,  0.0000]): 0
2 - tensor([ 3.0000,  1.0000, 74.0000,  0.0000,  0.0000,  7.7750,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 3.0000,  0.0000,  5.0000,  0.0000,  0.0000, 12.4750,  2.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
4 - tensor([ 1.0000,  0.0000, 60.0000,  1.0000,  0.0000, 75.2500,  0.0000, 87.5090,
         3.0000,  1.0000,  0.0000]): 1
5 - tensor([ 1.0000,  1.0000, 27.0000,  1.0000,  0.0000, 53.1000,  2.0000, 87.5090,
         2.0000,  1.0000,  0.0000]): 1
6 - tensor([ 3.0000,  0.0000, 36.0000,  0.0000,  2.0000, 22.3583,  0.0000, 13.3029,
         3.0000,  2.0000,  0.0000]): 1
7 - tensor([ 2.00

### 메인함수
1. 데이터셋 준비 및 크기 확인: 전처리된 데이터셋을 준비하고, 각각의 데이터셋 크기를 확인하는 방법.
2. DataLoader 사용: DataLoader를 사용해 데이터를 배치 단위로 로드하고, 학습 및 검증 과정에서 데이터를 처리하는 방법.
3. 배치 처리 및 형태 확인: 각 배치의 입력 데이터와 타겟 데이터를 확인하고, 이를 출력하는 과정을 통해 모델이 처리하는 데이터를 파악.
4. 모델 훈련 및 검증: 학습 데이터셋과 검증 데이터셋을 사용해 모델을 훈련하고, 각 배치 단위로 데이터를 처리하는 과정을 확인.
5. 테스트 수행: 학습된 모델을 테스트 데이터에 적용하고, 예측 결과를 확인하는 방법.
6. 데이터 흐름 및 출력 제어: 데이터 흐름을 시각적으로 구분하는 출력 방식을 사용하여, 학습과 테스트 과정을 명확하게 이해할 수 있는 방법.

In [261]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import pandas as pd
import sys
import argparse

# 데이터 로드 함수 (학습 및 검증 데이터 로드)
def get_train_val_data(batch_size):
    """
    학습 및 검증 데이터를 로드하고 DataLoader로 변환하여 반환하는 함수.

    :param batch_size: (int) 배치 사이즈.
    :return:
        tuple: 학습 데이터 로더 (DataLoader), 검증 데이터 로더 (DataLoader).
    """
    train_dataset, validation_dataset, _ = get_preprocessed_dataset()  # test 데이터는 무시
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
    return train_data_loader, validation_data_loader

### 학습 및 검증 데이터 로드
* 학습 및 검증을 위해 데이터를 로드한다
* 테스트 데이터는 성능 평가에 사용해야 하기에 무시한다.

In [262]:
# 테스트 데이터 로드 함수 (학습 및 검증 후에 사용)
def get_test_data():
    """
    테스트 데이터를 로드하고 DataLoader로 변환하여 반환하는 함수. 학습 및 검증 후 성능 평가에 사용.

    :return: DataLoader: 테스트 데이터 로더.
    """
    _, _, test_dataset = get_preprocessed_dataset()  # 학습 및 검증 데이터는 무시
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))
    return test_data_loader

### 테스트용 데이터 로드
* 성능평가에 사용할 데이터 로드

In [263]:
# 모델 정의 - 활성화 함수를 인자로 받아 신경망을 구성
class MyModel(nn.Module):
    """
    신경망 모델 정의 클래스.

    :param n_input: (int) 입력 특징 벡터의 크기.
    :param n_output: (int) 출력 노드 수 (분류 클래스의 수).
    :param activation_fn: (nn.Module) 사용할 활성화 함수.
    """

    def __init__(self, n_input, n_output, activation_fn):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),  # 첫 번째 은닉층
            activation_fn(),  # 활성화 함수
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),  # 두 번째 은닉층
            activation_fn(),  # 활성화 함수
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),  # 출력층
        )

    def forward(self, x):
        """
        순전파 함수 (Forward Propagation).

        :param x: (torch.Tensor) 입력 데이터.
        :return: (torch.Tensor) 출력 데이터.
        """
        return self.model(x)

### 모델 정의
* 여러 활성화 함수가 적용가능 하도록 함

In [264]:
# 모델과 옵티마이저 설정
def get_model_and_optimizer(activation_fn):
    """
    모델과 옵티마이저를 설정하는 함수.

    :param activation_fn: (nn.Module) 사용할 활성화 함수.
    :return:
        tuple: 신경망 모델 (MyModel), 옵티마이저 (SGD 옵티마이저).
    """
    model = MyModel(n_input=11, n_output=2, activation_fn=activation_fn)  # 입력은 11개 특징, 출력은 2개 (생존 여부)
    optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate)  # SGD 옵티마이저 사용
    return model, optimizer

### 모델, 옵티마이저 설정 함수
* 모델 구조와 하이퍼파라미터를 유연하게 설정 가능하다.
* 신경망 모델의 기본 구조: 입력층, 은닉층, 활성화 함수, 출력층

In [265]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader, test_data_loader=None):
    """
    주어진 에포크 수만큼 모델을 학습하고 검증하는 훈련 루프 함수.
    각 에포크마다 훈련 손실 및 정확도, 검증 손실 및 정확도를 기록하며,
    성능이 가장 좋은 모델 상태를 저장합니다.

    :param model: (nn.Module) 학습할 신경망 모델.
    :param optimizer: (torch.optim.Optimizer) 모델 학습을 위한 옵티마이저.
    :param train_data_loader: (DataLoader) 학습 데이터 로더.
    :param validation_data_loader: (DataLoader) 검증 데이터 로더.
    :param test_data_loader: (DataLoader, optional) 테스트 데이터 로더.
    :return: tuple (최종 검증 정확도, 최적 모델 상태, 최고 성능을 낸 epoch 번호)
    """
    
    # 설정된 에포크 수
    n_epochs = wandb.config.epochs

    # 손실 함수: 이진 분류를 위한 CrossEntropyLoss 사용
    loss_fn = nn.CrossEntropyLoss()

    # 최고 검증 정확도를 기록하기 위한 변수
    best_validation_accuracy = 0.0

    # 가장 성능이 좋은 모델 상태와 epoch를 저장할 변수
    best_model_state = None
    best_epoch = 0

    # 에포크 반복 시작
    for epoch in range(1, n_epochs + 1):
        model.train()  # 모델을 학습 모드로 전환
        loss_train = 0.0  # 학습 손실 초기화
        correct_train = 0  # 학습 정확도 계산을 위한 변수 초기화
        num_trains = 0  # 처리한 학습 데이터의 개수

        # 학습 데이터에 대해 배치 단위로 학습 진행
        for batch in train_data_loader:
            input = batch['input']  # 입력 데이터
            target = batch['target']  # 레이블 (목표값)

            # 모델에 입력 데이터를 넣어 예측값 계산
            output_train = model(input)

            # 손실 함수로 예측값과 레이블 간 손실 계산
            loss = loss_fn(output_train, target)
            loss_train += loss.item()  # 전체 손실 합계

            # 예측값의 클래스(생존 여부)를 선택하고 정확도를 계산
            prediction = torch.argmax(output_train, dim=1)
            correct_train += (prediction == target).sum().item()  # 정확도 계산
            num_trains += target.size(0)  # 처리한 데이터 개수

            # 옵티마이저 초기화 및 역전파로 가중치 업데이트
            optimizer.zero_grad()  # 기존 기울기 초기화
            loss.backward()  # 역전파 수행
            optimizer.step()  # 가중치 업데이트

        # 검증 단계
        model.eval()  # 모델을 평가 모드로 전환
        loss_validation = 0.0  # 검증 손실 초기화
        correct_validation = 0  # 검증 정확도 계산을 위한 변수 초기화
        num_validations = 0  # 검증 데이터 개수

        # 검증 데이터에 대해 성능 평가 (기울기 계산하지 않음)
        with torch.no_grad():
            for batch in validation_data_loader:
                input = batch['input']  # 검증 입력 데이터
                target = batch['target']  # 검증 레이블

                # 검증 데이터로 예측값 계산
                output_validation = model(input)

                # 검증 손실 계산
                loss = loss_fn(output_validation, target)
                loss_validation += loss.item()

                # 예측값의 클래스(생존 여부)를 선택하고 검증 정확도 계산
                prediction = torch.argmax(output_validation, dim=1)
                correct_validation += (prediction == target).sum().item()
                num_validations += target.size(0)

        # 검증 정확도 계산
        validation_accuracy = correct_validation / num_validations

        # 현재 에포크에서 검증 정확도가 이전보다 높다면 모델 상태를 저장
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_model_state = model.state_dict()  # 모델 상태 저장
            best_epoch = epoch  # 성능이 가장 좋은 에포크 저장

        # 학습과 검증 과정을 WandB에 로깅
        wandb.log({
            "Epoch": epoch,
            "Training Loss": loss_train / num_trains,  # 학습 손실
            "Training Accuracy": correct_train / num_trains,  # 학습 정확도
            "Validation Loss": loss_validation / num_validations,  # 검증 손실
            "Validation Accuracy": validation_accuracy  # 검증 정확도
        })

        # 매 100번째 에포크마다 검증 정확도 출력
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Validation Accuracy: {validation_accuracy:.4f}")

    # 가장 성능이 좋은 에포크와 검증 정확도를 출력
    print(f"Best model found at epoch {best_epoch} with validation accuracy {best_validation_accuracy:.4f}")

    # 최종적으로 최고 성능이 나온 모델 상태와 그때의 검증 정확도, 에포크 반환
    return best_validation_accuracy, best_model_state, best_epoch


### 모델 학습 및 검증 함수
* `nn.CrossEntropyLoss()` : 예측 값과 실제 레이블 간의 손실을 최소화하는 방향으로 학습함
* `torch.argmax(output_train, dim=1)`을 사용해 모델의 예측 결과를 분류하고, 이를 실제 값과 비교하여 정확도를 계산함.
* 최고성능을 기록하는 epcoh 시점을 기록해 그 순간 submisson.csv 파일을 생성한다.

In [266]:
# 테스트 루프 (성능 평가)
def test_model(model, test_data_loader):
    """
    학습된 모델을 사용하여 테스트 데이터에 대해 예측하는 함수.

    :param model: (nn.Module) 학습된 모델.
    :param test_data_loader: (DataLoader) 테스트 데이터 로더.
    :return: list: 테스트 데이터에 대한 예측 결과.
    """
    model.eval()  # 평가 모드로 전환
    predictions = []

    with torch.no_grad():
        for batch in test_data_loader:
            input = batch['input']
            output = model(input)
            prediction = torch.argmax(output, dim=1)  # 예측 값 계산
            predictions.extend(prediction.tolist())  # 예측 결과 리스트에 추가

    return predictions

### 성능 테스트 함수

In [267]:
# Submission 파일 생성
def create_submission(predictions, output_file="submission.csv"):
    """
    테스트 데이터 예측 결과를 사용해 Kaggle 제출용 submission 파일 생성하는 함수.

    :param predictions: (list) 예측 결과.
    :param output_file: (str) 생성할 submission 파일 이름.
    """
    submission = pd.DataFrame({
        "PassengerId": range(892, 892 + len(predictions)),
        "Survived": predictions
    })
    submission.to_csv(output_file, index=False)
    print(f"Submission file saved as {output_file}")

### submisson.csv 파일 생성

In [268]:
# 메인 함수
def main(args):
    """
    모델 학습, 검증 및 최종 평가를 수행하는 메인 함수.
    """
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [30, 30],
    }

    # Step 1: ReLU 활성화 함수로 학습
    wandb.init(
        mode="online",
        project="titanic_model_training_with_ReLU",
        notes="Titanic dataset classification with ReLU",
        tags=["titanic", "classification", "ReLU"],
        name=current_time_str,
        config=config
    )
    print("WandB initialized for ReLU")

    # 학습 및 검증 데이터 로드
    train_data_loader, validation_data_loader = get_train_val_data(wandb.config.batch_size)
    # 테스트 데이터 로드 (최종 선택된 모델로만 사용)
    test_data_loader = get_test_data()

    # ReLU 활성화 함수로 모델 학습
    model, optimizer = get_model_and_optimizer(nn.ReLU)
    relu_validation_accuracy, best_model_state, best_epoch = training_loop(
        model=model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )

    print(f"ReLU Validation Accuracy: {relu_validation_accuracy:.4f}")
    wandb.finish()

    # Step 2: 다른 활성화 함수들과 성능 비교
    activation_functions = {
        "ELU": nn.ELU,
        "Leaky ReLU": nn.LeakyReLU,
        "PReLU": nn.PReLU,
        "Mish": nn.Mish
    }

    best_activation_function = "ReLU"
    best_validation_accuracy = relu_validation_accuracy

    for activation_name, activation_fn in activation_functions.items():
        print(f"Starting experiment with {activation_name}")
        wandb.init(
            mode="online",
            project=f"titanic_model_training_with_{activation_name}",
            notes=f"Titanic dataset classification with {activation_name}",
            tags=["titanic", "classification", activation_name],
            name=f"{activation_name}_{current_time_str}",
            config=config
        )
        print(f"WandB initialized for {activation_name}")

        model, optimizer = get_model_and_optimizer(activation_fn)

        validation_accuracy, model_state, epoch = training_loop(
            model=model,
            optimizer=optimizer,
            train_data_loader=train_data_loader,
            validation_data_loader=validation_data_loader
        )

        print(f"{activation_name} Validation Accuracy: {validation_accuracy:.4f}")

        # 더 나은 검증 정확도 발견 시 갱신
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_activation_function = activation_name
            best_model_state = model_state
            best_epoch = epoch  # 최고 성능을 낸 epoch 저장

        wandb.finish()

    print(
        f"The best activation function is {best_activation_function} with a validation accuracy of {best_validation_accuracy:.4f} at epoch {best_epoch}")

    # Step 3: 최종 모델로 테스트 데이터에 대해 예측 및 최종 submission 파일 생성
    print(f"Testing the best model with {best_activation_function}")
    model.load_state_dict(best_model_state)  # 최적의 모델 상태 로드
    test_predictions = test_model(model, test_data_loader)  # 테스트 데이터 예측

    # 최종적으로 가장 성능이 좋은 epoch에서의 모델로 submission 파일 생성
    create_submission(test_predictions, output_file=f"submission_best_epoch_{best_epoch}.csv")
    print(f"Final submission file created for the best epoch: {best_epoch}")


### 요구사항 2
1. wandb 그래프 얻기
* [그래프 레포트](https://wandb.ai/marching12-korea-university-of-technology-and-education/titanic_model_training_with_Leaky%20ReLU/reports/Untitled-Report--Vmlldzo5ODM2NDU4/edit?draftId=Vmlldzo5ODM2NDU4&firstReport&runsetFilter)
2. 성능이 가장 좋은 활성화 함수 찾기
3. 가장 성능 좋은 활성화 함수로 모델 구성하여 submisson.csv 만들기
* 성능이 가장 좋은 시점의 epoch를 저장해 그 시점에 submisson.csv 파일을 만든다.

In [269]:
# 명령어 인자 처리
if __name__ == "__main__":
    """
    명령줄 인자 처리를 통해 훈련 설정을 받아들인 후 main 함수를 호출하여 학습을 시작하는 부분.
    """
    if 'ipykernel_launcher' in sys.argv[0]:
        sys.argv = [sys.argv[0]]  # Jupyter에서 실행될 때 인자를 무시
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 16)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=1000, help="Number of training epochs (int, default:1000)"
    )

    args = parser.parse_args()

    main(args)

Epoch,▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
Training Accuracy,▁▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇████▇
Training Loss,█▇█▇▇▇▇▇▇▇▆▆▅▅▅▅▅▅▄▅▄▅▅▅▄▄▄▄▃▃▃▂▂▂▃▂▁▁▁▁
Validation Accuracy,▃▂▂▃▁▂▂▄▃▁▄▃▂▂▄▂▄▂▅▅▂▅▅▅▄▃▆▇▃▆▆█▇▄▆▄███▆
Validation Loss,▇▇█▆▇▆▆▆▅▅▆▆▅▅▆▅▅▄▄▅▄▄▅▄▄▄▆▃▃▃▃▃▄▃▂▂▁▁▂▁
Epoch,240
Training Accuracy,0.73773
Training Loss,0.03413
Validation Accuracy,0.78652
Validation Loss,0.00278


WandB initialized for ReLU
Epoch 100: Validation Accuracy: 0.6854
Epoch 200: Validation Accuracy: 0.6854
Epoch 300: Validation Accuracy: 0.7022
Epoch 400: Validation Accuracy: 0.5955
Epoch 500: Validation Accuracy: 0.7247
Epoch 600: Validation Accuracy: 0.7640
Epoch 700: Validation Accuracy: 0.7360
Epoch 800: Validation Accuracy: 0.6180
Epoch 900: Validation Accuracy: 0.7584
Epoch 1000: Validation Accuracy: 0.7584
Best model found at epoch 564 with validation accuracy 0.7809
ReLU Validation Accuracy: 0.7809


Epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
Training Accuracy,▁▂▁▂▂▂▁▂▂▁▂▂▂▂▂▃▃▄▄▅▅▇▆▇▆▆▆▆▆▇▇█▇▇█▇▇▇██
Training Loss,███▇▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▂▃▂▂▃▂▂▂▁▂▂▂▂▁▂▁▁
Validation Accuracy,▃▄▄▅▃▄▄▅▅▄▃▇▇▇▆▄▆▇▃▇▇▄▇▇▇▇▆▇▇▇▁▅█▇▇▆▇▇█▇
Validation Loss,▆▆▆▆▆▆▆▆▆▆▅▅▄▄▄▂█▅▄▁▃▇▁▂▂▁▁▂▃▂▄▆▂▁▂▄▃▄▂▁
Epoch,1000
Training Accuracy,0.82889
Training Loss,0.02484
Validation Accuracy,0.75843
Validation Loss,0.00318


Starting experiment with ELU


WandB initialized for ELU
Epoch 100: Validation Accuracy: 0.6573
Epoch 200: Validation Accuracy: 0.6742
Epoch 300: Validation Accuracy: 0.6910
Epoch 400: Validation Accuracy: 0.6854
Epoch 500: Validation Accuracy: 0.7191
Epoch 600: Validation Accuracy: 0.7584
Epoch 700: Validation Accuracy: 0.7191
Epoch 800: Validation Accuracy: 0.7360
Epoch 900: Validation Accuracy: 0.7472
Epoch 1000: Validation Accuracy: 0.7416
Best model found at epoch 699 with validation accuracy 0.7697
ELU Validation Accuracy: 0.7697


...

Epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█
Training Accuracy,▁▂▂▂▃▃▃▂▂▃▃▃▅▅▅▆▇▇▆▇▇▆▇▇▇▆▇█▆▇█▇▇██▇▇▇██
Training Loss,████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▄▃▃▃▃▃▃▂▂▃▂▂▂▁▁▂▁▂▁▂▂
Validation Accuracy,▄▂▃▃▅▃▅▃▄▂▃▂▅▄▆▇█▇▇▇█▇█▇▁▆▇▆▆▇▇████▇▆▆▇▆
Validation Loss,▆▆▅▅▅▅▅▅▅▆▄▃▃▃█▅▃▂▂▄▂▁▄▁▃▁▄▁▂▁▂▂▃▃▂▁▄▂▃▁
Epoch,1000
Training Accuracy,0.82749
Training Loss,0.02502
Validation Accuracy,0.73596
Validation Loss,0.00294


Starting experiment with Mish


WandB initialized for Mish
Epoch 100: Validation Accuracy: 0.6742
Epoch 200: Validation Accuracy: 0.6854
Epoch 300: Validation Accuracy: 0.6629
Epoch 400: Validation Accuracy: 0.6854
Epoch 500: Validation Accuracy: 0.7809
Epoch 600: Validation Accuracy: 0.7303
Epoch 700: Validation Accuracy: 0.7584
Epoch 800: Validation Accuracy: 0.7360
Epoch 900: Validation Accuracy: 0.7416
Epoch 1000: Validation Accuracy: 0.7697
Best model found at epoch 500 with validation accuracy 0.7809
Mish Validation Accuracy: 0.7809


Epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
Training Accuracy,▁▁▁▂▂▂▃▂▂▃▃▃▃▄▄▄▅▅▆▅▇▆▇▇▇▆▆▇▇█▇█▇█▇▇▇▇▇▇
Training Loss,██▇▇▇▇▅▅▅▅▄▄▄▄▄▄▃▄▃▂▂▃▂▂▁▂▂▁▁▂▂▁▂▂▂▃▂▂▁▁
Validation Accuracy,▁▃▃▄▄▄▂▅▃▄▅▂▄▄▅▄▇▅▆▄▇▇▆███▇█▇▆████▇███▆█
Validation Loss,▆▆▅▅▅▅▅▅▄▄▄▃▄▄▃▂▄▃▅▂▂▇▃▂▃▂█▁▅▁▃▂▇▄▂▁▃▁▂▄
Epoch,1000
Training Accuracy,0.84432
Training Loss,0.02476
Validation Accuracy,0.76966
Validation Loss,0.00318


The best activation function is Leaky ReLU with a validation accuracy of 0.7921 at epoch 257
Testing the best model with Leaky ReLU
Submission file saved as submission_best_epoch_257.csv
Final submission file created for the best epoch: 257


### 메인함수 실행

### 숙제후기
공학설계를 진행하며 ai기술을 활용할 계획을 세웠는데 여러 모델학습의 과정을 실습해보며 공학설계를 진행하는데 있어 도움이 될 것 같다는 생각을 했습니다. 

![캐글등록](https://ibb.co/j66fB62"><img src="https://i.ibb.co/GxxpBxL/image.png" alt="image" border="0"></a>)